In [1]:
import numpy as np
import cv2
import glob
import imutils
from hessian import Hessian2D
from frangiFilter2D import eig2image,FrangiFilter2D
import progressbar

In [2]:
def ShowResizedIm(img,windowname,scale):
    cv2.namedWindow(windowname, cv2.WINDOW_NORMAL)        # Create window with freedom of dimensions
    height, width = img.shape[:2]   #get image dimension
    cv2.resizeWindow(windowname,int(width/scale) ,int(height/scale))                    # Resize image
    cv2.imshow(windowname, img)                            # Show image

In [37]:
#=========USER START================
#folder path
path = 'RAW_FUNDUS_INPUT/*.jpg'


#=========USER END================

In [39]:
image_list = []
for filename in glob.glob(path):
    image_list.append(filename)
with progressbar.ProgressBar(max_value=len(image_list)) as bar:
    progress = 0
    bar.update(progress)
    for i,a in enumerate(image_list):
        img = cv2.imread(a)
        b,g,r = cv2.split(img)
        blood = cv2.normalize(g.astype('double'), None, 0.0, 1.0, cv2.NORM_MINMAX) # Convert to normalized floating point
        outIm, whatScale, direction = FrangiFilter2D(blood,FrangiScaleRange=np.array([5, 10]), FrangiScaleRatio=2,
                           FrangiBetaOne=0.5, FrangiBetaTwo=15, verbose=True, BlackWhite=True)
        _,thresh = cv2.threshold(outIm*10000000,0.1,255,0)
        thresh = cv2.convertScaleAbs(thresh)
        kernel1 = np.ones((11,11),np.uint8)
        kernel2 = np.ones((5,5),np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel2)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel1)
        ROImask =  getMaskROI(r)
        closing = cv2.bitwise_and(closing,closing, mask=ROImask)
        cv2.imwrite("bv image/"+ a.replace(path.replace("/*.jpg","")+"\\",""),closing)
        progress = progress+1
        bar.update(progress)
        

100% (540 of 540) |######################| Elapsed Time: 0:07:25 Time:  0:07:25


KeyboardInterrupt: 

In [82]:
image_list = []
for filename in glob.glob(path):
    image_list.append(filename)

In [7]:
img = cv2.imread(image_list[0])
ShowResizedIm(np.hstack([img,]),"mark",2)
cv2.waitKey(0)
cv2.destroyAllWindows()

IndexError: list index out of range

In [6]:
b,g,r = cv2.split(img)
blood = cv2.normalize(g.astype('double'), None, 0.0, 1.0, cv2.NORM_MINMAX) # Convert to normalized floating point
outIm, whatScale, direction = FrangiFilter2D(blood,FrangiScaleRange=np.array([5, 10]), FrangiScaleRatio=2,
                   FrangiBetaOne=0.5, FrangiBetaTwo=15, verbose=True, BlackWhite=True)
ShowResizedIm(np.hstack([outIm,]),"out",2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
test = outIm.copy()*10000000
_,thresh = cv2.threshold(test,0.1,255,0)
threah = cv2.convertScaleAbs(thresh)
kernel1 = np.ones((11,11),np.uint8)
kernel2 = np.ones((5,5),np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel2)
closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel1)
ShowResizedIm(np.hstack([thresh,opening,closing]),"out",3)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [63]:
cv2.imwrite("bv image/"+ image_list[0].replace(path.replace("/*.jpg","")+"\\",""),closing)

True

In [26]:
mask = np.ones((thresh.shape[:2]), dtype=np.uint8)*255
__,contours, hierarchy = cv2.findContours(cv2.convertScaleAbs(thresh), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
if len(contours) != 0:
    c = max(contours, key = cv2.contourArea)
    cv2.drawContours(mask, [c], -1, 0,  -1)
    thresh = cv2.bitwise_and(thresh, thresh, mask=mask)
ShowResizedIm(np.hstack([thresh,mask]),"out",3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [79]:
__,cut_edge_mask = cv2.threshold(r,10,255,0)
cut_edge_mask = cv2.Canny(cut_edge_mask,100,200)
circles = cv2.HoughCircles(cut_edge_mask,cv2.HOUGH_GRADIENT,1,1000,
                            param1=2,param2=4,minRadius=800,maxRadius=1000)
r2 = r.copy()
if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
        print("hey")
        # draw the outer circle
        cv2.circle(g2,(i[0],i[1]),i[2],(255,255,255),2)
        # draw the center of the circle
        cv2.circle(g2,(i[0],i[1]),2,(255,255,255),3)

thresh = cv2.bitwise_and(thresh,thresh, mask=cut_edge_mask)
ShowResizedIm(np.hstack([r2,]),"out",3)
cv2.waitKey(0)
cv2.destroyAllWindows()

hey
hey
hey
hey
hey


In [67]:
ShowResizedIm(np.hstack([cut_edge_mask,]),"out",3)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
#=========USER START================
#folder path
path2 = 'error  image/*.jpg'


#=========USER END================

In [13]:
image_list = []
for filename in glob.glob(path2):
    image_list.append(filename)

In [25]:
with progressbar.ProgressBar(max_value=len(image_list)) as bar:
    progress = 0
    bar.update(progress)
    for i,a in enumerate(image_list):
        img = cv2.imread(a)
        b,g,r = cv2.split(img)
        __,cut_edge_mask = cv2.threshold(r,10,255,0)
        cut_edge_mask = cv2.Canny(cut_edge_mask,100,200)
        circles = cv2.HoughCircles(cut_edge_mask,cv2.HOUGH_GRADIENT,1,10000,
                                    param1=2,param2=4,minRadius=800,maxRadius=1000)
        if circles is not None:
            if len(circles) == 1:
                mask = np.zeros((r.shape[:2]), dtype=np.uint8)
                circles = np.uint16(np.around(circles))
                # draw the outer circle
                cv2.circle(mask,(circles[0][0][0],circles[0][0][1]),circles[0][0][2]-20,255,-1)
                # draw the center of the circle
                #cv2.circle(r2,(circles[0][0][0],circles[0][0][1]),2,(255,255,255),3)
        cv2.imwrite("cut edge image/"+ a.replace(path2.replace("/*.jpg","")+"\\",""),mask)
        progress = progress+1
        bar.update(progress)

100% (10 of 10) |########################| Elapsed Time: 0:00:05 Time:  0:00:05


In [36]:
def getMaskROI(r,minR=800,maxR=1000):
    __,cut_edge_mask = cv2.threshold(r,10,255,0)
    cut_edge_mask = cv2.Canny(cut_edge_mask,100,200)
    circles = cv2.HoughCircles(cut_edge_mask,cv2.HOUGH_GRADIENT,1,10000,
                                param1=2,param2=4,minRadius=minR,maxRadius=maxR)
    if circles is not None:
        if len(circles) == 1:
            mask = np.zeros((r.shape[:2]), dtype=np.uint8)
            circles = np.uint16(np.around(circles))
            # draw the outer circle
            cv2.circle(mask,(circles[0][0][0],circles[0][0][1]),circles[0][0][2]-20,255,-1)
            return mask
        else:
            print("error more than one ROI")
            return None
    else:
        print("error detecting ROI")
        return None